# Assignment # 5


##1. Automated File Management and Data Export System


In [1]:
import os
import glob
import shutil
import pandas as pd

In [2]:
os.makedirs("csv_files", exist_ok=True)
os.makedirs("backup_folder", exist_ok=True)

In [3]:
sample_data = pd.DataFrame({
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35],
    "City": ["New York", "Los Angeles", "Chicago"]
})

for i in range(1, 4):
    sample_data.to_csv(f"csv_files/sample_{i}.csv", index=False)

In [4]:
csv_files = glob.glob("csv_files/*.csv")

for file in csv_files:
    shutil.move(file, "backup_folder/")
    print(f"Moved file: {file}")


Moved file: csv_files/sample_1.csv
Moved file: csv_files/sample_2.csv
Moved file: csv_files/sample_3.csv


In [5]:
def export_data(df, filename, format):
    if format == "csv":
        df.to_csv(filename, index=False)
        print(f"Data exported to {filename} in CSV format.")
    elif format == "json":
        df.to_json(filename, orient="records")
        print(f"Data exported to {filename} in JSON format.")
    else:
        print("Unsupported format.")

df = pd.DataFrame(sample_data)

export_data(df, "output.csv", "csv")
export_data(df, "output.json", "json")


Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


##2. Real-Time Stock Market Data Collection and Analysis

In [6]:
import yfinance as yf
import sqlite3
import pandas as pd
import time

In [7]:
db_name = "stocks.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                symbol TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER)''')
conn.commit()

In [8]:
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None

        latest = data.iloc[-1]
        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

In [9]:
def store_data(symbol):
    stock_data = fetch_stock_data(symbol)
    if stock_data:
        cursor.execute('''INSERT INTO stock_data (symbol, open, high, low, close, volume)
                          VALUES (?, ?, ?, ?, ?, ?)''',
                       (stock_data["symbol"], stock_data["open"],
                        stock_data["high"], stock_data["low"],
                        stock_data["close"], stock_data["volume"]))
        conn.commit()
        print(f"Stored data for {symbol}")

In [11]:
def analyze_stock(symbol):
    df = pd.read_sql_query("SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100", conn, params=(symbol,))
    print(df)


In [26]:
symbol = input("Enter which stock you want to analyze").upper()
for _ in range(5):
    store_data(symbol)
    time.sleep(60)  # Wait for 1 minute before fetching again

Enter which stock you want to analyzenvda
Stored data for NVDA
Stored data for NVDA
Stored data for NVDA
Stored data for NVDA
Stored data for NVDA


In [27]:
analyze_stock(symbol)

   id symbol            timestamp        open        high         low  \
0  19   NVDA  2025-02-18 14:39:55  142.229706  142.229706  142.229706   
1  18   NVDA  2025-02-18 14:38:55  142.419998  142.419998  142.419998   
2  17   NVDA  2025-02-18 14:37:55  142.514999  142.514999  142.514999   
3  16   NVDA  2025-02-18 14:36:54  142.859894  142.859894  142.859894   
4  15   NVDA  2025-02-18 14:35:54  143.399994  143.399994  143.399994   

        close  volume  
0  142.229706       0  
1  142.419998       0  
2  142.514999       0  
3  142.859894       0  
4  143.399994       0  


In [28]:
conn.close()

##3. Augmented Reality Transformation – Perform linear algebra operations like scaling, rotation, and translation

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [30]:
URL = "https://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

In [31]:
def get_books(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    books = soup.find_all("article", class_="product_pod")
    book_list = []

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text
        stock = book.find("p", class_="instock availability").text.strip()

        book_list.append({"Title": title, "Price": price, "Availability": stock})

    return book_list

In [32]:
books_data = get_books(URL)

In [33]:
df = pd.DataFrame(books_data)
df.to_csv("books.csv", index=False)
print("Data saved to books.csv")

Data saved to books.csv
